In [1]:


from utils import  main 

main() 

/home/amir/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-10 17:43:44,944 - INFO - Loading articles from JSONL file...
2024-11-10 17:43:49,010 - INFO - Loaded 237381 articles
2024-11-10 17:43:49,010 - INFO - Preparing training data...
2024-11-10 17:43:49,467 - INFO - Preparing validation data...
2024-11-10 17:43:49,979 - INFO - Using device: cuda
/home/amir/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing samples: 100%|██████████| 1/1 [0


Processed 275 samples
Skipped 215 samples without citation


Processing samples: 100%|██████████| 1/1 [00:03<00:00,  3.00s/batch]
2024-11-10 17:43:57,131 - INFO - Starting training...



Processed 303 samples
Skipped 321 samples without citation

Epoch 1/10


Training:   0%|          | 0/9 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 23.59 GiB of which 2.88 MiB is free. Process 2456058 has 7.24 GiB memory in use. Process 2468937 has 7.37 GiB memory in use. Including non-PyTorch memory, this process has 8.94 GiB memory in use. Of the allocated memory 8.59 GiB is allocated by PyTorch, and 53.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)